# 注意メカニズムとトランスフォーマー

リカレントネットワークの主な欠点の一つは、シーケンス内のすべての単語が結果に同じ影響を与えることです。このため、名前付きエンティティ認識や機械翻訳などのシーケンス間タスクにおいて、標準的なLSTMエンコーダーデコーダーモデルでは性能が最適化されません。実際には、入力シーケンス内の特定の単語が他の単語よりも順序出力に大きな影響を与えることがよくあります。

機械翻訳のようなシーケンス間モデルを考えてみましょう。このモデルは2つのリカレントネットワークによって実装されます。一つのネットワーク（**エンコーダー**）が入力シーケンスを隠れ状態に圧縮し、もう一つのネットワーク（**デコーダー**）がその隠れ状態を展開して翻訳結果を生成します。このアプローチの問題点は、ネットワークの最終状態が文の冒頭を記憶するのが難しくなるため、長い文に対してモデルの品質が低下することです。

**注意メカニズム**は、RNNの各出力予測に対する各入力ベクトルの文脈的影響を重み付けする手段を提供します。このメカニズムは、入力RNNの中間状態と出力RNNの間にショートカットを作成することで実装されます。この方法では、出力記号$y_t$を生成する際に、異なる重み係数$\alpha_{t,i}$を用いてすべての入力隠れ状態$h_i$を考慮します。

![エンコーダー/デコーダーモデルと加法型注意層を示す画像](../../../../../translated_images/ja/encoder-decoder-attention.7a726296894fb567.webp)
*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf)の加法型注意メカニズムを備えたエンコーダーデコーダーモデル。画像は[このブログ記事](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)から引用。]*

注意行列$\{\alpha_{i,j}\}$は、出力シーケンス内の特定の単語の生成において、入力単語がどの程度影響を与えるかを表します。以下はそのような行列の例です：

![Bahdanau - arviz.orgから引用されたRNNsearch-50によるサンプルアラインメントを示す画像](../../../../../translated_images/ja/bahdanau-fig3.09ba2d37f202a6af.webp)

*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf)（図3）から引用された図]*

注意メカニズムは、現在またはほぼ現在の自然言語処理の最先端技術の多くを支えています。しかし、注意を追加することでモデルのパラメータ数が大幅に増加し、RNNのスケーリング問題を引き起こしました。RNNのスケーリングにおける重要な制約は、モデルのリカレント性がバッチ処理やトレーニングの並列化を困難にすることです。RNNではシーケンスの各要素を順序通りに処理する必要があるため、簡単に並列化することができません。

注意メカニズムの採用とこの制約が組み合わさり、現在私たちが使用している最先端のトランスフォーマーモデル（BERTやOpenGPT3など）が誕生しました。

## トランスフォーマーモデル

各予測の文脈を次の評価ステップに渡す代わりに、**トランスフォーマーモデル**は**位置エンコーディング**と**注意**を使用して、指定されたテキストウィンドウ内で与えられた入力の文脈を捉えます。以下の画像は、位置エンコーディングと注意がどのようにして指定されたウィンドウ内の文脈を捉えるかを示しています。

![トランスフォーマーモデルで評価がどのように行われるかを示すアニメーションGIF](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

各入力位置が独立して各出力位置にマッピングされるため、トランスフォーマーはRNNよりも並列化が容易であり、より大規模で表現力豊かな言語モデルを可能にします。各注意ヘッドは、単語間の異なる関係を学習するために使用され、自然言語処理の下流タスクを改善します。

## シンプルなトランスフォーマーモデルの構築

Kerasには組み込みのトランスフォーマーレイヤーは含まれていませんが、自分で構築することができます。これまでと同様に、AG Newsデータセットのテキスト分類に焦点を当てますが、トランスフォーマーモデルはより難しいNLPタスクで最良の結果を示すことに注意する価値があります。


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Kerasの新しいレイヤーは`Layer`クラスを継承し、`call`メソッドを実装する必要があります。まずは**Positional Embedding**レイヤーから始めましょう。[公式のKerasドキュメント](https://keras.io/examples/nlp/text_classification_with_transformer/)のコードを使用します。すべての入力シーケンスを長さ`maxlen`にパディングすることを前提とします。


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

このレイヤーは2つの`Embedding`レイヤーで構成されています。1つはトークンを埋め込むためのもので（前に説明した方法で行います）、もう1つはトークンの位置を埋め込むためのものです。トークンの位置は、`tf.range`を使用して0から`maxlen`までの自然数のシーケンスとして作成され、それを埋め込みレイヤーに渡します。この2つの埋め込みベクトルを加算することで、入力の位置情報を埋め込んだ表現が得られます。この表現の形状は`maxlen`$\times$`embed_dim`となります。

次に、トランスフォーマーブロックを実装してみましょう。これは、先ほど定義した埋め込みレイヤーの出力を受け取ります。


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Transformerは、位置エンコードされた入力に対して`MultiHeadAttention`を適用し、`maxlen`$\times$`embed_dim`の次元を持つアテンションベクトルを生成します。このベクトルは入力と混合され、`LayerNormalization`を使用して正規化されます。

> **Note**: `LayerNormalization`は、この学習パスの*コンピュータビジョン*の部分で説明した`BatchNormalization`に似ていますが、各トレーニングサンプルの前の層の出力を独立して正規化し、[-1..1]の範囲に収めます。

この層の出力は次に`Dense`ネットワーク（この場合、2層のパーセプトロン）を通過し、その結果が最終出力に加えられます（最終出力も再び正規化されます）。

さて、完全なTransformerモデルを定義する準備が整いました：


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT トランスフォーマーモデル

**BERT**（Bidirectional Encoder Representations from Transformers）は、非常に大規模な多層トランスフォーマーネットワークで、*BERT-base* では12層、*BERT-large* では24層の構造を持っています。このモデルは、まず大規模なテキストデータ（Wikipedia + 書籍）を使用して、教師なし学習（文中のマスクされた単語を予測する）で事前学習されます。事前学習の過程で、モデルは言語理解の高度な知識を吸収し、その後、他のデータセットで微調整を行うことで活用できます。このプロセスは**転移学習**と呼ばれます。

![http://jalammar.github.io/illustrated-bert/ からの画像](../../../../../translated_images/ja/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

BERT、DistilBERT、BigBird、OpenGPT3 など、微調整可能なトランスフォーマーアーキテクチャには多くのバリエーションがあります。

では、事前学習済みの BERT モデルを使用して、従来のシーケンス分類問題を解決する方法を見てみましょう。[公式ドキュメント](https://www.tensorflow.org/text/tutorials/classify_text_with_bert)からアイデアと一部のコードを拝借します。

事前学習済みモデルをロードするには、**Tensorflow hub** を使用します。まず、BERT 専用のベクトライザーをロードしましょう:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

元のネットワークが訓練された際に使用されたものと同じベクトライザーを使用することが重要です。また、BERTベクトライザーは以下の3つのコンポーネントを返します：
* `input_word_ids`：入力文のトークン番号のシーケンス
* `input_mask`：シーケンスのどの部分が実際の入力で、どの部分がパディングであるかを示します。これは、`Masking`レイヤーによって生成されるマスクに似ています
* `input_type_ids`：言語モデリングタスクで使用され、1つのシーケンス内で2つの入力文を指定することができます。

次に、BERT特徴抽出器をインスタンス化します：


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


BERT層は以下のような有用な結果を返します：
* `pooled_output` は、シーケンス内のすべてのトークンを平均化した結果です。これをネットワーク全体の知的な意味的埋め込みと見なすことができます。これは、以前のモデルで使用した `GlobalAveragePooling1D` 層の出力に相当します。
* `sequence_output` は、最後のトランスフォーマーレイヤーの出力です（上記のモデルで使用した `TransformerBlock` の出力に対応します）。
* `encoder_outputs` は、すべてのトランスフォーマーレイヤーの出力です。4層のBERTモデルをロードしているため（名前に `4_H` が含まれていることから推測できるように）、4つのテンソルがあります。最後のテンソルは `sequence_output` と同じです。

次に、エンドツーエンドの分類モデルを定義します。*関数型モデル定義* を使用し、モデルの入力を定義した後、一連の式を用いてその出力を計算します。また、BERTモデルの重みを学習可能にせず、最終的な分類器のみを学習させます：


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


訓練可能なパラメータが少ないにもかかわらず、プロセスはかなり遅いです。これは、BERTの特徴抽出が計算負荷が高いためです。訓練不足やモデルパラメータの不足が原因で、十分な精度を達成できなかったようです。

BERTの重みをアンフリーズして、それも訓練してみましょう。これには非常に小さい学習率が必要であり、**ウォームアップ**を伴うより慎重な訓練戦略が必要です。また、**AdamW**オプティマイザーを使用します。オプティマイザーを作成するために`tf-models-official`パッケージを使用します。


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

トレーニングはかなりゆっくり進むようですが、いくつかのエポック（5〜10）でモデルをトレーニングしてみて、以前使用したアプローチと比較して最良の結果が得られるかどうか試してみると良いでしょう。

## Huggingface Transformersライブラリ

Transformerモデルを使用するもう一つの非常に一般的で少し簡単な方法は、[HuggingFaceパッケージ](https://github.com/huggingface/)を利用することです。このパッケージは、さまざまなNLPタスクのためのシンプルな構築ブロックを提供します。TensorflowとPyTorchの両方で利用可能で、PyTorchはもう一つの非常に人気のあるニューラルネットワークフレームワークです。

> **Note**: Transformersライブラリの動作に興味がない場合は、このノートブックの最後までスキップしても構いません。これまで行った内容と本質的に異なるものは見られないからです。BERTモデルを異なるライブラリとかなり大きなモデルを使用してトレーニングする同じ手順を繰り返します。そのため、プロセスにはかなり長いトレーニングが含まれるので、コードをざっと見るだけでも良いでしょう。

では、[Huggingface Transformers](http://huggingface.co)を使用してこの問題をどのように解決できるか見てみましょう。


まず最初に使用するモデルを選ぶ必要があります。組み込みモデルに加えて、Huggingfaceには[オンラインモデルリポジトリ](https://huggingface.co/models)があり、コミュニティによって提供された多くの事前学習済みモデルを見つけることができます。これらのモデルはすべて、モデル名を指定するだけでロードして使用することができます。必要なバイナリファイルは自動的にダウンロードされます。

場合によっては、自分自身のモデルをロードする必要があることもあります。その場合、トークナイザーのパラメータ、モデルパラメータを含む`config.json`ファイル、バイナリウェイトなど、関連するすべてのファイルが含まれるディレクトリを指定することができます。

モデル名から、モデルとトークナイザーの両方をインスタンス化することができます。まずはトークナイザーから始めましょう:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer`オブジェクトには、テキストを直接エンコードするために使用できる`encode`関数が含まれています。


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

シーケンスをモデルに渡すのに適した方法でエンコードするために、トークナイザーを使用することもできます。例えば、`token_ids`、`input_mask`フィールドなどを含めることができます。また、`return_tensors='tf'`引数を指定することで、TensorFlowのテンソルを取得することもできます。


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

私たちの場合、事前学習済みのBERTモデルである`bert-base-uncased`を使用します。*Uncased*は、このモデルが大文字小文字を区別しないことを示しています。

モデルを訓練する際には、トークン化されたシーケンスを入力として提供する必要があるため、データ処理パイプラインを設計します。`tokenizer.encode`はPythonの関数であるため、前のユニットで使用した方法と同様に、`py_function`を使用して呼び出します。


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

今度は、`BertForSequenceClassification`パッケージを使用して実際のモデルをロードできます。これにより、最終的な分類器を含む、分類に必要なアーキテクチャがすでにモデルに備わっていることが保証されます。最終的な分類器の重みが初期化されていないため、モデルには事前トレーニングが必要であるという警告メッセージが表示されますが、それは完全に問題ありません。なぜなら、まさにこれからそれを行うからです！


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


`summary()`からわかるように、このモデルにはほぼ1億1000万のパラメーターが含まれています！おそらく、比較的小さなデータセットで単純な分類タスクを行いたい場合、BERTのベース層を訓練したくないでしょう。


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


さあ、トレーニングを始めましょう！

> **Note**: フルスケールのBERTモデルをトレーニングするには非常に時間がかかる可能性があります！そのため、最初の32バッチだけトレーニングを行います。これはモデルのトレーニング設定を示すためのものです。もしフルスケールのトレーニングを試したい場合は、`steps_per_epoch`と`validation_steps`のパラメータを削除して、時間がかかる準備をしてください！


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


もし、イテレーションの回数を増やし、十分に待機し、さらに複数のエポックでトレーニングを行えば、BERTによる分類が最高の精度をもたらすことが期待できます！これは、BERTがすでに言語の構造を非常によく理解しており、最終的な分類器を微調整するだけで済むからです。しかし、BERTは大規模なモデルであるため、トレーニング全体のプロセスには時間がかかり、かなりの計算能力（GPU、できれば複数）が必要です。

> **Note:** この例では、最も小さい事前学習済みのBERTモデルの1つを使用しています。より大きなモデルを使用すれば、さらに良い結果が得られる可能性があります。


## まとめ

このユニットでは、**トランスフォーマー**に基づいた非常に新しいモデルアーキテクチャについて学びました。これらをテキスト分類タスクに適用しましたが、同様にBERTモデルはエンティティ抽出、質問応答、その他のNLPタスクにも使用できます。

トランスフォーマーモデルは、現在のNLPにおける最先端技術を代表しており、ほとんどの場合、カスタムNLPソリューションを実装する際に最初に試すべきアプローチです。しかし、より高度なニューラルモデルを構築したい場合、このモジュールで説明したリカレントニューラルネットワークの基本的な原理を理解することは非常に重要です。



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書を正式な情報源としてお考えください。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解釈について、当方は責任を負いません。
